In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 479.9 kB/s eta 0:00:00m eta 0:00:0136m0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 14.5 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 821.4 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 2.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 8.0 MB/s eta 0:00:000m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.6 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 936.2 kB/s eta 0:00:00 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 14.0 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 24.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

NameError: name 'device' is not defined

In [21]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
device = torch.device('cuda')

net = net.to(device)

def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (images, labels) in enumerate(data_loader):
            
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted_labels = torch.max(outputs, 1)
        num_examples += labels.size(0)
        correct_pred += (predicted_labels == labels).sum()
    return correct_pred.float() / num_examples * 100
    
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0
            
    # Compute accuracy after each epoch
    net.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        accuracy = compute_accuracy(net, testloader, device)
        print('Accuracy {}'.format(accuracy))

print('Finished Training')

[1,    20] loss: 1.281
[1,    40] loss: 1.334
[1,    60] loss: 1.335
[1,    80] loss: 1.329
[1,   100] loss: 1.326
[1,   120] loss: 1.348
[1,   140] loss: 1.321
[1,   160] loss: 1.328
[1,   180] loss: 1.334
[1,   200] loss: 1.277
[1,   220] loss: 1.321
[1,   240] loss: 1.303
[1,   260] loss: 1.310
[1,   280] loss: 1.284
[1,   300] loss: 1.335
[1,   320] loss: 1.312
[1,   340] loss: 1.289
[1,   360] loss: 1.328
[1,   380] loss: 1.334
Accuracy 52.679996490478516
[2,    20] loss: 1.266
[2,    40] loss: 1.307
[2,    60] loss: 1.300
[2,    80] loss: 1.287
[2,   100] loss: 1.295
[2,   120] loss: 1.316
[2,   140] loss: 1.314
[2,   160] loss: 1.285
[2,   180] loss: 1.261
[2,   200] loss: 1.278
[2,   220] loss: 1.299
[2,   240] loss: 1.316
[2,   260] loss: 1.265
[2,   280] loss: 1.316
[2,   300] loss: 1.326
[2,   320] loss: 1.273
[2,   340] loss: 1.311
[2,   360] loss: 1.290
[2,   380] loss: 1.296
Accuracy 53.8599967956543
[3,    20] loss: 1.259
[3,    40] loss: 1.264
[3,    60] loss: 1.262
[3,

In [12]:
testdataiter = iter(te

True